# part0: imports

In [ ]:
import os, sys, pathlib
from pprint import pprint 
from importlib import reload
import logging
from typing import Callable
import warnings



import pandas as pd
import numpy as np
import xarray as xr
from sklearn.decomposition import PCA
import scipy.linalg as linalg

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from matplotlib.ticker import MaxNLocator


import pyaldata as pyal
from defs import *

%matplotlib inline

try:
    nbPath = pathlib.Path.cwd()
    RepoPath = nbPath.parent
    os.chdir(RepoPath)

    from tools import utilityTools as utility
    from tools import dataTools as dt
    import params
    set_rc =  params.set_rc_params
    root = params.root

finally:
    os.chdir(nbPath)

#### Print the summary of all the datassets

check out the results [here](https://github.com/AtMostafa/notebook/blob/main/2021-cca-project/dataset-summary.md).

In [ ]:
_AllAnimalList = ['Chewie', 'Chewie2', 'Mihili', 'Jaco', "Chips", "MrT", 'Han', 'Lando']

_AllAnimalFiles=[]
for animal in _AllAnimalList:
    _AllAnimalFiles.extend(utility.find_file(root / animal, 'mat'))

# for i,fname in enumerate(_AllAnimalFiles):
#     _df = pyal.mat2dataframe(fname, shift_idx_fields=True)
#     print(i)
#     dt.summary(_df)

Categories I want to separate:

- sessions with 2 areas, similar number of neurons each, as many monkeys as possible!
- 3 sessions for each area (M1, PMd, S1), most number of neurons

In [ ]:
_AllAnimalFiles= [path.split(os.sep)[-1] for path in _AllAnimalFiles]

In [ ]:
GoodDataList = {'dualArea':{}, 'M1':{}, 'PMd':{}, 'S1':{}}

#------------------------------------

GoodDataList['dualArea']['Chewie'] = [_AllAnimalFiles[13],
                                      _AllAnimalFiles[12],
                                      _AllAnimalFiles[3],
                                      _AllAnimalFiles[1],
                                      _AllAnimalFiles[10]
                                     ]

GoodDataList['dualArea']['Mihili'] = [_AllAnimalFiles[64],
                                      _AllAnimalFiles[57],
                                      _AllAnimalFiles[68],
                                      _AllAnimalFiles[69]
                                     ]

GoodDataList['dualArea']['MrT'] = [_AllAnimalFiles[105],
                                   _AllAnimalFiles[103]
                                  ]

#-----------------------------------

GoodDataList['M1']['Chewie'] = [_AllAnimalFiles[13],
                                _AllAnimalFiles[10],
                                _AllAnimalFiles[12],
                                _AllAnimalFiles[1]
                               ]

GoodDataList['M1']['Chewie2'] = [_AllAnimalFiles[28],
                                 _AllAnimalFiles[22],
                                 _AllAnimalFiles[36],
                                 _AllAnimalFiles[47]
                                ]

GoodDataList['M1']['Mihili'] = [_AllAnimalFiles[64],
                                _AllAnimalFiles[57],
                                _AllAnimalFiles[68]
                               ]

GoodDataList['M1']['Jaco'] = [_AllAnimalFiles[85],
                              _AllAnimalFiles[87],
                              _AllAnimalFiles[81]
                             ]

#-----------------------------------

GoodDataList['PMd']['Chewie'] = [_AllAnimalFiles[9],
                                _AllAnimalFiles[13],
                                _AllAnimalFiles[8],
                                _AllAnimalFiles[11]
                               ]

GoodDataList['PMd']['Mihili'] = [_AllAnimalFiles[56],
                                 _AllAnimalFiles[74],
                                 _AllAnimalFiles[68]
                                ]

GoodDataList['PMd']['MrT'] = [_AllAnimalFiles[103],
                              _AllAnimalFiles[105],
                              _AllAnimalFiles[102]
                             ]


#------------------------------------

GoodDataList['S1']['Chips'] = [_AllAnimalFiles[95],
                               _AllAnimalFiles[97],
                               _AllAnimalFiles[94],
                               _AllAnimalFiles[92]
                              ]

GoodDataList['S1']['Han'] = [_AllAnimalFiles[119],
                             _AllAnimalFiles[107],
                             _AllAnimalFiles[116],
                             _AllAnimalFiles[115]
                            ]

GoodDataList['S1']['Lando'] = [_AllAnimalFiles[120],
                             _AllAnimalFiles[121],
                             _AllAnimalFiles[122]
                            ]


print(f'the variable `GoodDataList` contains the session names')

The code above relies on the host PC having the exact fiels as my laptop.  
So I replace the values below for reliability.

In [ ]:
GoodDataList = {'dualArea':{}, 'M1':{}, 'PMd':{}, 'S1':{}}

#------------------------------------

GoodDataList['dualArea']['Chewie'] = ['Chewie_CO_VR_2016-09-14.mat',
                                      'Chewie_CO_CS_2016-10-21.mat',
                                      'Chewie_CO_FF_2016-10-05.mat',
                                      'Chewie_CO_CS_2016-10-14.mat',
                                      'Chewie_CO_FF_2016-10-13.mat'
                                     ]

GoodDataList['dualArea']['Mihili'] = ['Mihili_CO_VR_2014-03-06.mat',
                                      'Mihili_CO_VR_2014-03-03.mat',
                                      'Mihili_CO_FF_2014-02-17.mat',
                                      'Mihili_CO_VR_2014-03-04.mat'
                                     ]

GoodDataList['dualArea']['MrT'] = ['MrT_CO_VR_2013-09-09.mat',
                                   'MrT_CO_VR_2013-09-05.mat'
                                  ]

#-----------------------------------

GoodDataList['M1']['Chewie'] = ['Chewie_CO_VR_2016-09-14.mat',
                                'Chewie_CO_FF_2016-10-13.mat',
                                'Chewie_CO_CS_2016-10-21.mat',
                                'Chewie_CO_CS_2016-10-14.mat'
                               ]

GoodDataList['M1']['Chewie2'] = ['Chewie_CO_CS_2015-03-19.mat',
                                 'Chewie_CO_CS_2015-03-13.mat',
                                 'Chewie_CO_CS_2015-03-11.mat',
                                 'Chewie_CO_CS_2015-03-12.mat'
                                ]

GoodDataList['M1']['Mihili'] = ['Mihili_CO_VR_2014-03-06.mat',
                                'Mihili_CO_VR_2014-03-03.mat',
                                'Mihili_CO_FF_2014-02-17.mat'
                               ]

GoodDataList['M1']['Jaco'] = ['Jaco_CO_CS_2016-01-28.mat',
                              'Jaco_CO_CS_2016-02-04.mat',
                              'Jaco_CO_CS_2016-02-17.mat'
                             ]

#-----------------------------------

GoodDataList['PMd']['Chewie'] = ['Chewie_CO_FF_2016-09-21.mat',
                                 'Chewie_CO_VR_2016-09-14.mat',
                                 'Chewie_CO_FF_2016-09-15.mat',
                                 'Chewie_CO_FF_2016-09-19.mat'
                                ]

GoodDataList['PMd']['Mihili'] = ['Mihili_CO_FF_2014-02-18.mat',
                                 'Mihili_CO_CS_2014-09-29.mat',
                                 'Mihili_CO_FF_2014-02-17.mat'
                                ]

GoodDataList['PMd']['MrT'] = ['MrT_CO_VR_2013-09-05.mat',
                              'MrT_CO_VR_2013-09-09.mat',
                              'MrT_CO_FF_2013-08-21.mat'
                             ]

#-----------------------------------

GoodDataList['S1']['Chips'] = ['Chips_20151204_TD_nosort_notrack_noemg.mat',
                               'Chips_20151201_TD_nosort_notrack_noemg.mat',
                               'Chips_20151211_TD_nosort_notrack_noemg.mat',
                               'Chips_20170505_TD_nosort_notrack_noemg.mat'
                              ]

GoodDataList['S1']['Han'] = ['Han_COactpas_2017-10-31.mat',
                             'Han_COactpas_2017-12-04.mat',
                             'Han_COactpas_2017-10-24.mat',
                             'Han_COactpas_2017-11-28.mat'
                            ]

GoodDataList['S1']['Lando'] = ['Lando_20170731_TD_nosort_notrack_noemg.mat',
                               'Lando_20170917_TD_nosort_notrack_noemg.mat',
                               'Lando_20170803_TD_nosort_notrack_noemg.mat'
                              ]
